<a href="https://colab.research.google.com/github/tilly963963/pytorch/blob/main/Chapter4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch

In [ ]:
torch.cuda.device(0)

In [ ]:
torch.cuda.get_device_name()

'Tesla T4'

In [ ]:
torch.cuda.is_available()

True

In [ ]:
num_epochs = 5
batch_size=100
learing_rate = 0.001

In [ ]:
import torchvision.datasets as dsets
import torchvision.transforms as transforms

In [ ]:
train_dataset = dsets.MNIST(root = './data',
              train = True,
              transform = transforms.ToTensor(),#把資料傳換
              download = True)#預設False
test_dataset = dsets.MNIST(root = './data',
              train = False,
              transform = transforms.ToTensor())#把資料傳換
               

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
import torch.utils.data as Data

In [ ]:
train_loader = Data.DataLoader(dataset = train_dataset,
              batch_size = batch_size,
              shuffle = True)
test_loader = Data.DataLoader(dataset = test_dataset,
              batch_size = batch_size,
              shuffle = False)

In [ ]:
import torch.nn as nn

In [ ]:
class Model(nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self.conv = nn.Sequential(
        nn.Conv2d(1,64,kernel_size=3,stride=1,padding=1),#input channel,outpuut channel=filter, 
        #輸出大小不變
        nn.ReLU(),
        nn.Conv2d(64, 128, kernel_size=3, padding=1),
        nn.MaxPool2d(stride=2,kernel_size=2),#通常步伐與kernel_size會設一樣大小
        nn.ReLU()
    )
    self.dense = nn.Sequential(
        nn.Linear(14*14*128, 1024),#大小*input_channel
        nn.ReLU(),
        nn.Linear(1024, 10)
    )
  def forward(self,x):
    out = self.conv(x)
    out = out.view(-1,14*14*128)
    out = self.dense(out)
    return out

In [ ]:
model = Model()
model.cuda()

Model(
  (conv): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): ReLU()
  )
  (dense): Sequential(
    (0): Linear(in_features=25088, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=10, bias=True)
  )
)

In [ ]:
import torch.optim as opt

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = opt.Adam(model.parameters(),lr=learing_rate)

In [ ]:
from torch.autograd import Variable

In [ ]:
for epoch in range(num_epochs):
  for i ,(images,labels) in enumerate(train_loader):
    images = Variable(images.type(torch.cuda.FloatTensor))
    labels = Variable(labels.type(torch.cuda.LongTensor))
    optimizer.zero_grad()
    outputs = model(images)
    loss = loss_fn(outputs, labels)
    loss.backward()
    optimizer.step()
    if (i+1)%100 == 0:
      print("Epoch:%d, Batch:%d , Loss:%0.4f"
      %(epoch+1,i+1,loss.data))

Epoch:1, Batch:100 , Loss:0.1301
Epoch:1, Batch:200 , Loss:0.0578
Epoch:1, Batch:300 , Loss:0.0606
Epoch:1, Batch:400 , Loss:0.0498
Epoch:1, Batch:500 , Loss:0.0552
Epoch:1, Batch:600 , Loss:0.0172
Epoch:2, Batch:100 , Loss:0.0168
Epoch:2, Batch:200 , Loss:0.0407
Epoch:2, Batch:300 , Loss:0.0194
Epoch:2, Batch:400 , Loss:0.0320
Epoch:2, Batch:500 , Loss:0.0103
Epoch:2, Batch:600 , Loss:0.0958
Epoch:3, Batch:100 , Loss:0.0024
Epoch:3, Batch:200 , Loss:0.0049
Epoch:3, Batch:300 , Loss:0.0239
Epoch:3, Batch:400 , Loss:0.0400
Epoch:3, Batch:500 , Loss:0.0185
Epoch:3, Batch:600 , Loss:0.0069
Epoch:4, Batch:100 , Loss:0.0039
Epoch:4, Batch:200 , Loss:0.0019
Epoch:4, Batch:300 , Loss:0.0105
Epoch:4, Batch:400 , Loss:0.0035
Epoch:4, Batch:500 , Loss:0.0110
Epoch:4, Batch:600 , Loss:0.0034
Epoch:5, Batch:100 , Loss:0.0003
Epoch:5, Batch:200 , Loss:0.0329
Epoch:5, Batch:300 , Loss:0.0011
Epoch:5, Batch:400 , Loss:0.0004
Epoch:5, Batch:500 , Loss:0.0059
Epoch:5, Batch:600 , Loss:0.0486


In [ ]:
import torch

In [ ]:
correct = 0 
total = 0
for images, labels in test_loader:
  images = Variable(images.type(torch.cuda.FloatTensor))
  outputs = model(images)
  _ ,pred = torch.max(outputs.data,1)#最大值 ,最大值的位置
  correct +=(pred == labels.type(torch.cuda.LongTensor)).sum()
  total += labels.size(0)
print("Accuracy:%.3f%%"%(100.0*float(correct)/float(total)))

Accuracy:99.150%
